<a href="https://colab.research.google.com/github/ramaastra/sekarya-machine-learning/blob/main/sekarya_model_with_kfold.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sekarya Machine Learning Model (Development)

This notebook containing some experiments of machine learning model development for Sekarya's main feature, to classify wheter an artwork is generated by AI or not.

For the model itself, the training will be done with a custom dataset we've collected below. For addition to the imbalanced ai_generated class in the dataset, we also add more data from Kaggle.

- [Sekarya Dataset](https://drive.google.com/drive/folders/1W_DN02xlxOB9M_P27_TJJNuhdxEQY9Kf?usp=drive_link)
- [Addition AI-generated Dataset](https://www.kaggle.com/datasets/gauravduttakiit/dalle-recognition-dataset)

## Preparing the Dataset

### Getting the Sekarya Dataset from Google Drive

In [1]:
from google.colab import drive
import os

drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!ls drive/MyDrive/New-Sekarya-Dataset/

test  train


In [3]:
drive_dataset_dir = '/content/drive/MyDrive/New-Sekarya-Dataset/'
os.listdir(drive_dataset_dir)

['train', 'test']

In [4]:
drive_train_dir = os.path.join(drive_dataset_dir, 'train')
os.listdir(drive_train_dir)

['ai_generated', 'non_ai_generated']

In [5]:
drive_train_fake_dir = os.path.join(drive_train_dir, 'ai_generated')
drive_train_real_dir = os.path.join(drive_train_dir, 'non_ai_generated')

print(f'There are {len(os.listdir(drive_train_fake_dir))} images of fake (AI-generated) artworks for training.\n')
print(f'There are {len(os.listdir(drive_train_real_dir))} images of real (human-made artworks) images for training.\n')

There are 331 images of fake (AI-generated) artworks for training.

There are 2501 images of real (human-made artworks) images for training.



### Getting Additional Dataset from Kaggle

In [6]:
from google.colab import files
import os

if not(os.path.exists("kaggle.json")):
  files.upload()
!pip install --upgrade --force-reinstall --no-deps kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!ls ~/.kaggle
!chmod 600 /root/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.6/83.6 kB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for kaggle: filename=kaggle-1.5.16-py3-none-any.whl size=110683 sha256=91d269d0ac6493c03c5399e499008e09381f81d03dd5af51f216ea735aae87ee
  Stored in directory: /root/.cache/pip/wheels/43/4b/fb/736478af5e8004810081a06259f9aa2f7c3329fc5d03c2c412
Successfully built kaggle
  Attempting uninstall: kaggle
    Found existing installation: kaggle 1.5.16
    Uninstalling kaggle-1.5.16:
      Successfully uninstalled kaggle-1.5.16
kaggle.json


In [7]:
dataset_url = 'gauravduttakiit/dalle-recognition-dataset'
dataset_name = dataset_url.split('/')[1]
!kaggle datasets download -d  {dataset_url}
!mkdir {dataset_name}
!unzip -q {dataset_name}.zip -d {dataset_name}
!rm -f {dataset_name}.zip

100% 1.35G/1.35G [00:55<00:00, 23.5MB/s]
100% 1.35G/1.35G [00:55<00:00, 26.3MB/s]


In [8]:
kaggle_train_fake_dir = f'/content/{dataset_name}/train/fake'
print(f'There are {len(os.listdir(kaggle_train_fake_dir))} additional images of fake (AI-generated) artworks for training.\n')

There are 2057 additional images of fake (AI-generated) artworks for training.



### Storing Data File Paths and Labels to a List

In [9]:
images = []
labels = []

# Dataset from Google Drive
class_labels = os.listdir(drive_train_dir)
for class_label in class_labels:
  class_data_path = os.path.join(drive_train_dir, class_label)
  for filename in os.listdir(class_data_path):
    file_path = os.path.join(class_data_path, filename)
    images.append(file_path)
    labels.append(class_label)

# Additional AI-generated class data from Kaggle
for filename in os.listdir(kaggle_train_fake_dir):
  file_path = os.path.join(kaggle_train_fake_dir, filename)
  images.append(file_path)
  labels.append('ai_generated')

print(f'There are {len(images)} images will be splitted with K-fold cross-validation technique.\n')

There are 4889 images will be splitted with K-fold cross-validation technique.



## Preparing the Model Architecture

---



In [10]:
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import StratifiedKFold
import pandas as pd
import numpy as np
import random

### Defining the CNN Model

In [11]:
def cnn_model():
  model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(16, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid'),
  ])

  model.compile(optimizer=Adam(learning_rate=1e-4),
                loss=BinaryCrossentropy(),
                metrics=['accuracy'])

  return model

### Defining EarlyStopping Callback

In [12]:
early_stopping = EarlyStopping(monitor='val_loss',
                               patience=5,
                               mode='min',
                               restore_best_weights=True)

## Training the Model with K-fold Cross-validation

In [16]:
model = cnn_model()

In [17]:
num_folds = 5
kfold = StratifiedKFold(n_splits=num_folds)

In [18]:
accuracies = []
losses = []

for k, (train_indices, val_indices) in enumerate(kfold.split(images, labels)):
  print(f'[Processing Fold-{k}...]\n')

  # Creating lists for images and labels based on the train and val indices
  x_train = [images[i] for i in train_indices]
  y_train = [labels[i] for i in train_indices]
  x_val = [images[i] for i in val_indices]
  y_val = [labels[i] for i in val_indices]

  # Creating dataframe for each train and val list
  train_df = pd.DataFrame({
    'image': x_train,
    'label': y_train
  })
  val_df = pd.DataFrame({
    'image': x_val,
    'label': y_val
  })

  # Creating the image generator to process the images
  train_datagen = ImageDataGenerator(rescale=1./255.0)
  val_datagen = ImageDataGenerator(rescale=1./255.0)

  train_generator = train_datagen.flow_from_dataframe(train_df,
                                                      x_col='image',
                                                      y_col='label',
                                                      target_size=(224, 224),
                                                      batch_size=64,
                                                      color_mode='rgb',
                                                      class_mode='binary')

  val_generator = val_datagen.flow_from_dataframe(val_df,
                                                  x_col='image',
                                                  y_col='label',
                                                  target_size=(224, 224),
                                                  batch_size=64,
                                                  color_mode='rgb',
                                                  class_mode='binary')

  print()

  # Train the model for this fold
  history = model.fit(train_generator,
                      epochs=10,
                      validation_data=val_generator,
                      verbose=1,
                      callbacks=[early_stopping])

  # Evaluate the model on the validation set
  _, accuracy = model.evaluate(val_generator)
  print(f'\nValidation accuracy for fold-{k}: {accuracy:.4f}')

  # Store the accuracy and loss for this fold
  accuracies.append(accuracy)
  losses.append(history.history['loss'][-1])

  print('==============================================================\n\n')

[Processing Fold-0...]

Found 3911 validated image filenames belonging to 2 classes.
Found 978 validated image filenames belonging to 2 classes.

Epoch 1/10
62/62 [==============================] - 86s 1s/step - loss: 0.6771 - accuracy: 0.5664 - val_loss: 0.5915 - val_accuracy: 0.7198
Epoch 2/10
62/62 [==============================] - 89s 1s/step - loss: 0.5974 - accuracy: 0.6694 - val_loss: 0.5160 - val_accuracy: 0.8282
Epoch 3/10
62/62 [==============================] - 81s 1s/step - loss: 0.5151 - accuracy: 0.7622 - val_loss: 0.4427 - val_accuracy: 0.7587
Epoch 4/10
62/62 [==============================] - 79s 1s/step - loss: 0.4700 - accuracy: 0.7811 - val_loss: 0.3641 - val_accuracy: 0.8640
Epoch 5/10
62/62 [==============================] - 79s 1s/step - loss: 0.4532 - accuracy: 0.7888 - val_loss: 0.3438 - val_accuracy: 0.8712
Epoch 6/10
62/62 [==============================] - 78s 1s/step - loss: 0.4073 - accuracy: 0.8215 - val_loss: 0.3418 - val_accuracy: 0.8671
Epoch 7/10
62/

In [21]:
# Save the model into models directory
model.save('/content/models/model.h5')

In [22]:
# Calculate average accuracy and loss across folds
average_accuracy = sum(accuracies) / num_folds
average_loss = sum(losses) / num_folds

print(f'Average accuracy: {average_accuracy:.4f}')
print(f'Average loss: {average_loss:.4f}')

Average accuracy: 0.9552
Average loss: 0.1091
